<a href="https://colab.research.google.com/github/ManagementBC/SMO/blob/main/AgenticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leveraging a Multi-LLM Agentic Framework for Evidence-Guided Second Medical Opinion Generation**

In [ ]:
# === 📦 Install Required Libraries (Google Colab) ===
!pip install -q langchain faiss-cpu gradio python-dotenv PyPDF2 tiktoken langchain-community
!pip install -U langchain-openai langchain-anthropic langchain-google-genai

# === 🌐 Load Environment Variables ===
import os
from dotenv import load_dotenv
load_dotenv()

# If needed, manually set API keys (for Colab)
os.environ["OPENAI_API_KEY"] = "xxxxxx"
os.environ["ANTHROPIC_API_KEY"] = "xxxxxx"
os.environ["GOOGLE_API_KEY"] = "xxxxxx"

# Confirm keys are loaded
print("✅ OpenAI:", bool(os.getenv("OPENAI_API_KEY")))
print("✅ Anthropic:", bool(os.getenv("ANTHROPIC_API_KEY")))
print("✅ Google:", bool(os.getenv("GOOGLE_API_KEY")))


In [ ]:
# === LLM Initialization ===
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

llm_openai = ChatOpenAI(model="gpt-4o", temperature=0.2)
llm_claude = ChatAnthropic(model="claude-3-opus-20240229", temperature=0.2)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)


In [ ]:
# === 🔽 Upload Guideline PDFs (for Colab) ===
!pip install pypdf
from google.colab import files
uploaded = files.upload()  # Upload WHO + MSF PDFs here

# === 🧾 Parse PDFs ===
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

msf_loader = PyPDFLoader("Respiratory diseases - MSF MEDICAL GUIDLINES.pdf")
who_loader = PyPDFLoader("TUV_D1_RESPIRATORY GUIDELINES WHO.pdf")

msf_docs = msf_loader.load()
who_docs = who_loader.load()

# Add metadata source tags
for doc in msf_docs:
    doc.metadata["source"] = "MSF"
for doc in who_docs:
    doc.metadata["source"] = "WHO"

all_docs = msf_docs + who_docs
print(f"✅ Loaded {len(msf_docs)} MSF docs, {len(who_docs)} WHO docs")

# === 🔪 Split Text into Chunks for Vector Embedding ===
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = text_splitter.split_documents(all_docs)
print(f"✅ Created {len(chunks)} vector chunks")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 18.7 MB/s eta 0:00:00


Saving Respiratory diseases - MSF MEDICAL GUIDLINES.pdf to Respiratory diseases - MSF MEDICAL GUIDLINES.pdf
Saving TUV_D1_RESPIRATORY GUIDELINES WHO.pdf to TUV_D1_RESPIRATORY GUIDELINES WHO.pdf
✅ Loaded 52 MSF docs, 54 WHO docs
✅ Created 310 vector chunks


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Save to local directory
vectorstore.save_local("faiss_guidelines_index")
print("✅ Vectorstore saved to: faiss_guidelines_index")


/tmp/ipython-input-10-4188840010.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


✅ Vectorstore saved to: faiss_guidelines_index


In [ ]:
def retrieve_guideline_context(symptoms: str, k: int = 3) -> str:
    """
    Search vector DB for most relevant guideline chunks.
    """
    results = vectorstore.similarity_search(symptoms, k=k)
    seen = set()
    filtered = []

    for doc in results:
        chunk = doc.page_content.strip()
        if chunk and chunk not in seen:
            filtered.append(chunk)
            seen.add(chunk)

    return "\n\n".join(filtered[:3])  # Return top-3 unique chunks


In [ ]:
def get_latest_visit_for_patient(patient_id):
    """
    Retrieves the most recent visit document for a given patient ID
    from the FAISS vector store.
    """
    matches = [
        doc for doc in vectorstore.docstore._dict.values()
        if doc.metadata.get("visit_id") == patient_id
    ]

    if not matches:
        return None

    # Sort by timestamp if available, fallback to order
    matches.sort(key=lambda doc: doc.metadata["visit_json"].get("timestamp", ""), reverse=True)

    return matches[0].page_content


In [ ]:
import re

def extract_patient_metadata(text):
    pid = re.search(r"Patient ID:\s*(\S+)", text)
    vtype = re.search(r"Visit Type:\s*(\S+)", text)
    return (
        pid.group(1).strip() if pid else None,
        vtype.group(1).strip().lower() if vtype else None
    )



In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
import re

# === GPT-4 Tool ===
def openai_rag_tool(input_text: str) -> str:
    prev_visit = None
    patient_id, visit_type = extract_patient_metadata(input_text)

    if visit_type == "follow-up" and patient_id:
        prev_visit = get_latest_visit_for_patient(patient_id)
        if prev_visit:
            input_text = f"""This is a follow-up visit. Compare the current condition with the previous one. Evaluate if symptoms have improved, worsened, or remained unchanged. Adjust the treatment plan accordingly.

== Previous Visit ==
{prev_visit}

== Current Visit ==
{input_text}
"""

    rag = retrieve_guideline_context(input_text)

    prompt = f"""
You are a medical expert following WHO and MSF respiratory guidelines.

{input_text}

== Retrieved Guidelines ==
{rag}

Return a structured response with:
1. Is the condition improving, worsening, or unchanged if it is a follow up? but If this is a new case with no previous data, say: "Not applicable – first recorded visit."
2. Likely Diagnosis
3. Severity Classification
4. Updated Treatment Plan
"""
    return llm_openai.invoke(prompt).content

# === Claude Tool ===
def claude_rag_tool(input_text: str) -> str:
    prev_visit = None
    patient_id, visit_type = extract_patient_metadata(input_text)

    if visit_type == "follow-up" and patient_id:
        prev_visit = get_latest_visit_for_patient(patient_id)
        if prev_visit:
            input_text = f"""This is a follow-up visit. Compare the patient's condition today with the previous visit. Indicate any changes in symptoms, diagnosis, or severity. If treatment has failed, recommend next steps.

== Previous Visit ==
{prev_visit}

== Current Visit ==
{input_text}
"""

    rag = retrieve_guideline_context(input_text)

    prompt = f"""
You are a medical expert following WHO and MSF respiratory guidelines.

{input_text}

== Retrieved Guidelines ==
{rag}

Return a structured medical opinion:
1. Is the condition improving, worsening, or unchanged if it is a follow up? but If this is a new case with no previous data, say: "Not applicable – first recorded visit."
2. Diagnosis
3. Severity
4. Treatment Plan Adjustments
"""
    return llm_claude.invoke(prompt).content

# === Gemini Tool ===
def gemini_rag_tool(input_text: str) -> str:
    prev_visit = None
    patient_id, visit_type = extract_patient_metadata(input_text)

    if visit_type == "follow-up" and patient_id:
        prev_visit = get_latest_visit_for_patient(patient_id)
        if prev_visit:
            input_text = f"""This is a follow-up visit. The patient has received previous treatment. Please compare this visit to the last one and determine if the symptoms are improving. Adjust your diagnosis and treatment plan accordingly.

== Previous Visit ==
{prev_visit}

== Current Visit ==
{input_text}
"""

    rag = retrieve_guideline_context(input_text)

    prompt = f"""
You are a medical expert following WHO and MSF respiratory guidelines.

{input_text}

== Retrieved Guidelines ==
{rag}

Output a structured response:
1. Is the condition improving, worsening, or unchanged if it is a follow up? but If this is a new case with no previous data, say: "Not applicable – first recorded visit."
2. Likely Diagnosis
3. Severity Classification
4. Recommended Treatment Plan
"""
    return llm_gemini.invoke(prompt).content


tools = [
    Tool(
        name="GPT-4 Respiratory Analyst",
        func=openai_rag_tool,
        description="Expert medical reasoning on respiratory cases using WHO/MSF"
    ),
    Tool(
        name="Claude 3 Respiratory Analyst",
        func=claude_rag_tool,
        description="xpert medical reasoning on respiratory cases using WHO/MSF"
    ),
    Tool(
        name="Gemini Respiratory Analyst",
        func=gemini_rag_tool,
        description="xpert medical reasoning on respiratory cases using WHO/MSF"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm_openai,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def run_agentic_combined_rag_synthesis(visit_type, patient_id, age, weight, symptoms, spo2, heart_rate, history):
    case = (
        f"Patient ID: {patient_id}\n"
        f"Visit Type: {visit_type}\n"
        f"Age: {age}, Weight: {weight} kg\n"
        f"Symptoms: {symptoms}\n"
        f"SpO₂: {spo2}%, Heart Rate: {heart_rate} bpm\n"
        f"Medical History: {history}"
    )

    prompt = f"""
You are a medical agent that can access expert reasoning tools.

Your job is to analyze the following patient case and return your final opinion in this exact 3-point format:

1. Is the condition improving, worsening, or unchanged if it is a follow up? but If this is a new case with no previous data, say: "Not applicable – first recorded visit."
2. Likely Diagnosis
3. Severity Classification
4. Recommended Treatment Plan

Do not always rely on the same tool. Consider the strengths of GPT-4, Claude, and Gemini for each individual case.


== Patient Case ==
{case}

Use your tools as needed. Your final answer must follow the required format.
"""

    try:
        response = agent.run(prompt)
    except Exception as e:
        response = f"❌ Agent execution failed: {e}"

    return response


In [ ]:
import gradio as gr
import uuid
from datetime import datetime

# === Global patient tracker ===
last_visit_data = {}

def handle_patient_input(visit_type, patient_id_input, age, weight, symptoms, spo2, heart_rate, history):
    global last_visit_data

    # Generate or reuse patient ID
    patient_id = patient_id_input.strip() if patient_id_input else f"mso-{uuid.uuid4().hex[:8]}"
    timestamp = datetime.now().isoformat()

    last_visit_data = {
        "patient_id": patient_id,
        "timestamp": timestamp,
        "age": age,
        "weight": weight,
        "symptoms": symptoms,
        "spo2": spo2,
        "heart_rate": heart_rate,
        "history": history,
        "visit_type": visit_type
    }

    summary = f"""### ✅ Visit Summary

**Patient ID:** `{patient_id}`
**Timestamp:** {timestamp}

**Age:** {age}
**Weight:** {weight} kg
**Symptoms:** {symptoms}
**SpO₂ Level:** {spo2}%
**Heart Rate:** {heart_rate} bpm
**Medical History:** {history}
"""
    return summary

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧠 Agentic Multi-LLM System for Second Medical Opinion Generation")

    with gr.Group():
        with gr.Row():
            patient_id = gr.Textbox(label="Patient ID (leave blank if new)")
            visit_type = gr.Radio(["New", "Follow-up"], label="Visit Type")

        with gr.Row():
            age = gr.Number(label="Age")
            weight = gr.Number(label="Weight (kg)")

        with gr.Row():
            spo2 = gr.Number(label="SpO₂ (%)")
            heart_rate = gr.Number(label="Heart Rate (bpm)")

        history = gr.Textbox(label="Medical History", placeholder="e.g., Asthma, recent viral infection")
        symptoms = gr.Textbox(label="Symptoms", placeholder="e.g., Cough, wheezing, shortness of breath")

        submit_btn = gr.Button("🧠 Get Agentic Medical Opinion")

    visit_summary = gr.Textbox(label="Visit Summary", lines=6, interactive=False)
    agentic_output = gr.Textbox(label="🧠 Final Agentic Opinion", lines=12, interactive=False)

    # Bind form and analysis logic
    submit_btn.click(
        fn=lambda *args: (
            handle_patient_input(*args),
            run_agentic_combined_rag_synthesis(*args)
        ),
        inputs=[visit_type, patient_id, age, weight, symptoms, spo2, heart_rate, history],
        outputs=[visit_summary, agentic_output]
    )

demo.launch(debug=True)